In [1]:
import inspect
from berny import Berny, geomlib, coords,Geometry,Math
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np
import ipyvolume as ipv
from numpy.linalg import norm as norm
from scipy.optimize import  minimize
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
from pyscf.geomopt.berny_solver import to_berny_geom as tbg
ang2bohr=1.8897261246
bohr2ang=.5291772109
np.set_printoptions(precision=3)

In [2]:
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")

In [3]:
from Morse import mpa
from AP_class import APDFT_perturbator as APP

In [4]:
def anglecenter(coords):
    return np.around(np.arccos(coords[1].dot(coords[2])/norm(coords[1])/norm(coords[2])),decimals=3)

In [5]:
h2o=gto.M(atom="O 0 0 0; H 0 -1.4 1.2; H 0 1.4 1.2",unit="Bohr",basis="def2-TZVP")

In [6]:
mf=scf.RHF(h2o)
E0=mf.scf()
g0=mf.Gradients().grad()
h0=mf.Hessian().hess()

converged SCF energy = -76.0561607707061
--------------- RHF gradients ---------------
         x                y                z
0 O    -0.0000000000    -0.0000000000    -0.0573469482
1 H    -0.0000000000    -0.0166425867     0.0286734741
2 H     0.0000000000     0.0166425867     0.0286734741
----------------------------------------------


In [7]:
h0=h0.swapaxes(1,2)

In [8]:
h0=h0.reshape(9,9)

In [9]:
h20_eq_geom=(optimize(mf).atom_coords())


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   0.000000  -0.740848   0.635013    0.000000  0.000000  0.000000
   H   0.000000   0.740848   0.635013    0.000000  0.000000  0.000000
converged SCF energy = -76.0561607707072
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 O     0.0000000000    -0.0000000000    -0.0573472742
1 H    -0.0000000000    -0.0166426445     0.0286736371
2 H     0.0000000000     0.0166426445     0.0286736371
----------------------------------------------
cycle 1: E = -76.0561607707  dE = -76.0562  norm(grad) = 0.0740744

Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000  -0.000000   0.038973    0.000000 -0.000000  0.038973
   H   0.000000  -0.746867   0.615526    0.00

In [278]:
h20_eq_geom-=h20_eq_geom[0]
minimum=anglecenter(h20_eq_geom),norm(h20_eq_geom[1])
anglecenter(h20_eq_geom),norm(h20_eq_geom[1]),norm(h20_eq_geom[2])

(1.859, 1.780170233145828, 1.7801702331458211)

In [11]:
h2occords=h2o.atom_coords()

In [12]:
geom2=tbg(h2o)

In [13]:
anglecenter(h2occords),norm(h2occords[1])

(1.724, 1.8439088914585773)

In [14]:
b2=Berny(geom2)
s=b2._state

In [15]:
s.coords._coords,vars(s.coords._coords[0])

([Bond(0, 1, weak=0), Bond(0, 2, weak=0), Angle(1, 0, 2, weak=0)],
 {'i': 0, 'j': 1, 'idx': (0, 1), 'weak': 0})

In [16]:
s.H  #use to get second derivs 

array([[0.508, 0.   , 0.   ],
       [0.   , 0.508, 0.   ],
       [0.   , 0.   , 0.191]])

In [17]:
geom2.coords*ang2bohr

array([[ 0. ,  0. ,  0. ],
       [ 0. , -1.4,  1.2],
       [ 0. ,  1.4,  1.2]])

In [18]:
b2._state.future.q  # bohr and radians

array([1.844, 1.844, 1.724])

In [19]:
q0=s.future.q

In [20]:
print(s.coords.B_matrix(s.geom))

[[ 0.     0.759 -0.651  0.    -0.759  0.651  0.     0.     0.   ]
 [ 0.    -0.759 -0.651  0.     0.     0.     0.     0.759  0.651]
 [ 0.     0.     0.824  0.    -0.353 -0.412  0.     0.353 -0.412]]


In [21]:
gsc=b2._state.geom.supercell()

In [22]:
vars(gsc),vars(s.geom) 

({'species': ['O', 'H', 'H'], 'coords': array([[ 0.   ,  0.   ,  0.   ],
         [ 0.   , -0.741,  0.635],
         [ 0.   ,  0.741,  0.635]]), 'lattice': None},
 {'species': ['O', 'H', 'H'], 'coords': array([[ 0.   ,  0.   ,  0.   ],
         [ 0.   , -0.741,  0.635],
         [ 0.   ,  0.741,  0.635]]), 'lattice': None})

In [23]:
[print(x) for x in enumerate(s.coords)]

(0, Bond(0, 1, weak=0))
(1, Bond(0, 2, weak=0))
(2, Angle(1, 0, 2, weak=0))


[None, None, None]

In [24]:
B=s.coords.B_matrix(s.geom)

In [25]:
for i,x in enumerate(s.coords):
    print(x.eval(gsc.coords, grad=True)[1],"\n \n")

[array([ 0.   ,  0.759, -0.651]), array([-0.   , -0.759,  0.651])] 
 

[array([ 0.   , -0.759, -0.651]), array([-0.   ,  0.759,  0.651])] 
 

[array([-0.   , -0.353, -0.412]), array([0.   , 0.   , 0.824]), array([-0.   ,  0.353, -0.412])] 
 



In [26]:
B

array([[ 0.   ,  0.759, -0.651,  0.   , -0.759,  0.651,  0.   ,  0.   ,
         0.   ],
       [ 0.   , -0.759, -0.651,  0.   ,  0.   ,  0.   ,  0.   ,  0.759,
         0.651],
       [ 0.   ,  0.   ,  0.824,  0.   , -0.353, -0.412,  0.   ,  0.353,
        -0.412]])

In [27]:
s.coords.eval_geom(s.geom)

array([1.844, 1.844, 1.724])

In [28]:
s.coords

<InternalCoords "bonds: 2, angles: 1, dihedrals: 0">

In [29]:
vars(coords.InternalCoords(s.geom, dihedral=s.params['dihedral'], superweakdih=s.params['superweakdih']))

{'_coords': [Bond(0, 1, weak=0), Bond(0, 2, weak=0), Angle(1, 0, 2, weak=0)],
 'fragments': [[0, 1, 2]]}

In [30]:
vars(coords.InternalCoords(s.geom, dihedral=s.params['dihedral'], superweakdih=s.params['superweakdih'])._coords[0])

{'i': 0, 'j': 1, 'idx': (0, 1), 'weak': 0}

In [31]:
vars(s.geom)

{'species': ['O', 'H', 'H'], 'coords': array([[ 0.   ,  0.   ,  0.   ],
        [ 0.   , -0.741,  0.635],
        [ 0.   ,  0.741,  0.635]]), 'lattice': None}

In [32]:
B.T

array([[ 0.   ,  0.   ,  0.   ],
       [ 0.759, -0.759,  0.   ],
       [-0.651, -0.651,  0.824],
       [ 0.   ,  0.   ,  0.   ],
       [-0.759,  0.   , -0.353],
       [ 0.651,  0.   , -0.412],
       [ 0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.759,  0.353],
       [ 0.   ,  0.651, -0.412]])

In [33]:
s.future.q
(B.T@s.future.q).reshape(3,3)#-(B.T@s.future.q).reshape(3,3)[0]

array([[ 0.   ,  0.   , -0.98 ],
       [ 0.   , -2.009,  0.49 ],
       [ 0.   ,  2.009,  0.49 ]])

In [34]:
s.geom.coords*ang2bohr

array([[ 0. ,  0. ,  0. ],
       [ 0. , -1.4,  1.2],
       [ 0. ,  1.4,  1.2]])

### praticamente qua mi d una proiezione dei modi normali sulle coordinate cartesiane della molecola, tutti insieme formano la B_matrix

In [35]:
a=np.array([-0.        , -0.91914503,  0.3939193 ])

In [36]:
norm(a)

1.0000000005430953

In [37]:
B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))

In [38]:
np.dot(B_inv.T, g0.reshape(-1))

array([ 0.031,  0.031, -0.02 ])

In [39]:
s.H

array([[0.508, 0.   , 0.   ],
       [0.   , 0.508, 0.   ],
       [0.   , 0.   , 0.191]])

I can have the projection of the gradient in the internal coordinates as well as the approximated hessian matrix.

In [40]:
proj = np.dot(B, B_inv)
proj

array([[ 1.000e+00, -3.331e-16,  1.110e-16],
       [-4.510e-17,  1.000e+00, -1.110e-16],
       [-2.776e-17,  6.939e-17,  1.000e+00]])

In [41]:
H_proj = proj.dot(s.H).dot(proj) + 1000 * (np.eye(len(s.coords)) - proj)
1000 * (np.eye(len(s.coords)) - proj)  #perhaps to avoid zeros in matrix inversion

array([[-4.441e-13,  3.331e-13, -1.110e-13],
       [ 4.510e-14,  0.000e+00,  1.110e-13],
       [ 2.776e-14, -6.939e-14,  1.110e-13]])

In [42]:
H_proj,s.H#differiscono per una manciata di infinesimi

(array([[ 5.079e-01,  3.327e-13, -1.109e-13],
        [ 4.506e-14,  5.079e-01,  1.109e-13],
        [ 2.774e-14, -6.934e-14,  1.910e-01]]), array([[0.508, 0.   , 0.   ],
        [0.   , 0.508, 0.   ],
        [0.   , 0.   , 0.191]]))

In [43]:
def quadratic_step(g, H, w, trust):
    ev = np.linalg.eigvalsh((H + H.T) / 2)
    rfo = np.vstack((np.hstack((H, g[:, None])), np.hstack((g, 0))[None, :]))
    D, V = np.linalg.eigh((rfo + rfo.T) / 2)
    dq = V[:-1, 0] / V[-1, 0]
    l = D[0]
    print(dq,l,ev)
    if norm(dq) <= trust:
        print('Pure RFO step was performed:')
        on_sphere = False
    else:
        def steplength(l):
            return norm(np.linalg.solve(l * np.eye(H.shape[0]) - H, g)) - trust
        l = Math.findroot(steplength, ev[0])  # minimization on sphere
        dq = np.linalg.solve(l * np.eye(H.shape[0]) - H, g)
        on_sphere = True
        print('Minimization on sphere was performed:')
    dE = np.dot(g, dq) + 0.5 * dq.dot(H).dot(dq)
    return dq, dE, on_sphere

In [44]:
np.hstack((g0.flatten(), 0))

array([-5.608e-17, -3.731e-16, -5.735e-02, -1.943e-16, -1.664e-02,
        2.867e-02,  2.504e-16,  1.664e-02,  2.867e-02,  0.000e+00])

In [45]:
current = Berny.Point(s.future.q, mf.e_tot, np.dot(B_inv.T, np.asarray(g0).reshape(-1)))

In [46]:
current.g # also np.dot(B_inv.T, np.asarray(g0).reshape(-1)) is g in internal coords

array([ 0.031,  0.031, -0.02 ])

In [47]:
 dq, dE, on_sphere = quadratic_step(np.dot(proj, current.g), H_proj, s.weights, s.trust)

[-0.061 -0.061  0.102] -0.005879406708602918 [0.191 0.508 0.508]
Pure RFO step was performed:


In [48]:
dq,current.q,dq+current.q,

(array([-0.061, -0.061,  0.102]),
 array([1.844, 1.844, 1.724]),
 array([1.783, 1.783, 1.827]))

In [49]:
-np.linalg.inv(H_proj)@(np.dot(proj, current.g))

array([-0.062, -0.062,  0.106])

In [50]:
np.linalg.solve(H_proj,np.dot(proj, current.g))

array([ 0.062,  0.062, -0.106])

In [51]:
dq # change dq to the one from Morse potential fitting 

array([-0.061, -0.061,  0.102])

In [52]:
current.q[0]

1.8439088914585773

## from dq in internal coords to dq in cartesian coords

In [53]:
q, a = s.coords.update_geom(s.geom, current.q, dq, B_inv)

In [54]:
q

array([1.783, 1.783, 1.827])

In [55]:
a.coords

array([[ 0.000e+00, -8.277e-15,  3.897e-02],
       [ 0.000e+00, -7.469e-01,  6.155e-01],
       [ 0.000e+00,  7.469e-01,  6.155e-01]])

In [56]:
(B.T@q).reshape(3,3)

array([[ 0.000e+00, -3.442e-14, -8.163e-01],
       [ 0.000e+00, -1.998e+00,  4.082e-01],
       [ 0.000e+00,  1.998e+00,  4.082e-01]])

In [57]:
b=a.coords
norm(b[1]-b[0]) #is angstrom

0.9435171386555323

In [58]:
B = s.coords.B_matrix(s.geom)
B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))
g_ic=np.dot(B_inv.T, (g0).reshape(-1))

In [59]:
B.T

array([[ 0.   ,  0.   ,  0.   ],
       [ 0.759, -0.759,  0.   ],
       [-0.651, -0.651,  0.824],
       [ 0.   ,  0.   ,  0.   ],
       [-0.759,  0.   , -0.353],
       [ 0.651,  0.   , -0.412],
       [ 0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.759,  0.353],
       [ 0.   ,  0.651, -0.412]])

In [60]:
B_inv@np.dot(B, B.T)

array([[ 0.000e+00,  0.000e+00,  0.000e+00],
       [ 7.593e-01, -7.593e-01,  4.994e-17],
       [-6.508e-01, -6.508e-01,  8.235e-01],
       [ 0.000e+00,  0.000e+00,  0.000e+00],
       [-7.593e-01,  3.331e-16, -3.529e-01],
       [ 6.508e-01, -1.943e-16, -4.118e-01],
       [ 0.000e+00,  0.000e+00,  0.000e+00],
       [ 1.110e-16,  7.593e-01,  3.529e-01],
       [-8.327e-17,  6.508e-01, -4.118e-01]])

In [61]:
B_inv.T@B.T@g_ic,g_ic

(array([ 0.031,  0.031, -0.02 ]), array([ 0.031,  0.031, -0.02 ]))

In [62]:
g_ic,g0

(array([ 0.031,  0.031, -0.02 ]), array([[-5.608e-17, -3.731e-16, -5.735e-02],
        [-1.943e-16, -1.664e-02,  2.867e-02],
        [ 2.504e-16,  1.664e-02,  2.867e-02]]))

In [63]:
np.allclose(B.T@g_ic,g0.flatten())

True

In [64]:
B@h2occords.flatten()   # problems with angles

array([1.844e+00, 1.844e+00, 1.110e-16])

In [65]:
s.future

Point(q=array([1.844, 1.844, 1.724]), E=None, g=None)

In [66]:
B

array([[ 0.   ,  0.759, -0.651,  0.   , -0.759,  0.651,  0.   ,  0.   ,
         0.   ],
       [ 0.   , -0.759, -0.651,  0.   ,  0.   ,  0.   ,  0.   ,  0.759,
         0.651],
       [ 0.   ,  0.   ,  0.824,  0.   , -0.353, -0.412,  0.   ,  0.353,
        -0.412]])

In [67]:
h2occords

array([[ 0. ,  0. ,  0. ],
       [ 0. , -1.4,  1.2],
       [ 0. ,  1.4,  1.2]])

In [68]:
-0.956/ 0.294*.4,0.956**2+ 0.294**2

(-1.3006802721088437, 1.000372)

$$B_{\mu i}=\frac{\partial q_\mu}{\partial x_i}$$

In [69]:
g_ic=np.dot(B_inv.T, (g0).reshape(-1))
g_ic

array([ 0.031,  0.031, -0.02 ])

In [70]:
h_ic=B_inv.T@h0@B_inv

In [138]:
h2o1=gto.M(atom="O 0 0 0; H 0 -1.4014 1.2012; H 0 1.4 1.2",unit="Bohr",basis="def2-TZVP")
h2o2=gto.M(atom="O 0 0 0; H 0 -1.3986 1.1988; H 0 1.4 1.2",unit="Bohr",basis="def2-TZVP")
E1=scf.RHF(h2o1).scf()
E2=scf.RHF(h2o2).scf()
(E2+E1-2*E0)/(.0014**2+.0012**2)

converged SCF energy = -76.0561022205584
converged SCF energy = -76.0562176330778


0.4964046993069026

In [72]:
B_inv.T@B.T@h_ic@B@B_inv,h_ic

(array([[ 0.501, -0.004,  0.037],
        [-0.004,  0.501,  0.037],
        [ 0.037,  0.037,  0.214]]), array([[ 0.501, -0.004,  0.037],
        [-0.004,  0.501,  0.037],
        [ 0.037,  0.037,  0.214]]))

In [73]:
B@B_inv

array([[ 1.000e+00, -3.331e-16,  1.110e-16],
       [-4.510e-17,  1.000e+00, -1.110e-16],
       [-2.776e-17,  6.939e-17,  1.000e+00]])

In [274]:
norm(h0),norm(h0-B.T@h_ic@B) #in realtà la traasformazione non è invertibile, ma ci si può avvicinare intorno al minimo.

(1.4325922553846953, 0.09040974238592206)

In [271]:
dq_pr=-np.linalg.inv(h_ic)@g_ic

In [272]:
np.set_printoptions(precision=3)

In [276]:
s.future.q+dq_pr

array([1.772, 1.772, 1.858])

In [245]:
s.future.q+quadratic_step(g_ic, h_ic, s.weights, s.trust)[0]

[-0.07  -0.07   0.128] -0.006975212719315234 [0.179 0.495 0.504]
Pure RFO step was performed:


array([1.774, 1.774, 1.852])

In [246]:
pred_q=(s.future.q-np.linalg.inv(h_ic)@g_ic)
pred_q

array([1.772, 1.772, 1.858])

In [247]:
qqq,gn=s.coords.update_geom(s.geom,q0,-np.linalg.inv(h_ic)@g_ic,B_inv)

In [248]:
gn=gn.coords*ang2bohr

In [249]:
gn-=gn[0]
norm(gn[2]),anglecenter(gn)

(1.7724385353302758, 1.858)

In [250]:
coords_new = h2occords+B_inv.dot(dq_pr).reshape(-1, 3)
coords_new-=coords_new[0]

In [251]:
norm(coords_new[2]),anglecenter(coords_new)

(1.7767002080583343, 1.863)

In [252]:
h2occords-(np.linalg.inv(h0)@g0.flatten()).reshape(3,3)

array([[-2.793e-15, -2.483e-02,  9.698e-02],
       [ 1.545e-14, -1.442e+00,  1.167e+00],
       [-2.379e-14,  1.392e+00,  1.167e+00]])

In [253]:
dxcc=-(np.linalg.inv(h0)@g0.flatten()).reshape(3,3)

In [254]:
dxcc-=dxcc[0]

In [255]:
norm((h2occords+dxcc)[1]),anglecenter(h2occords+dxcc)

(1.7754409637512851, 1.848)

In [256]:
q0,q0+dq_pr

(array([1.844, 1.844, 1.724]), array([1.772, 1.772, 1.858]))

In [257]:
h_ic,g_ic,dq_pr

(array([[0.496, 0.001, 0.032],
        [0.001, 0.496, 0.032],
        [0.032, 0.032, 0.185]]),
 array([ 0.031,  0.031, -0.02 ]),
 array([-0.071, -0.071,  0.133]))

In [258]:
-g_ic[0]/h_ic[0,0],dq_pr[0]

(-0.06304673043868449, -0.07147069552881277)

In [260]:
app=APP(mf,sites=[1])
app.build_all()

In [261]:
app.APDFT3(np.asarray([-1]))-app.mf.e_tot

0.6890273872543986

In [262]:
mpa(g_ic[0],h_ic[0,0],s.future.q[0],.674)[0]-s.future.q[0],dq_pr[0],-g_ic[0]/h_ic[0,0]

(-0.05943921716876921, -0.07147069552881277, -0.06304673043868449)

In [270]:
ddq_mb=np.asarray([mpa(g_ic[0],h_ic[0,0],s.future.q[0],.67)[0]+g_ic[0]/h_ic[0,0]-s.future.q[0],\
                mpa(g_ic[0],h_ic[0,0],s.future.q[0],.67)[0]+g_ic[0]/h_ic[0,0]-s.future.q[0],0])
dq_mb0=dq_pr+ddq_mb
ddq_mb

(array([0.004, 0.004, 0.   ]), array([-0.068, -0.068,  0.133]))

In [277]:
mpa(g_ic[0],h_ic[0,0],s.future.q[0],.674)[0],\
mpa(g_ic[0],h_ic[0,0],s.future.q[0],.674)[0]+g_ic[0]/h_ic[0,0]+dq_pr[0],s.future.q[0]+dq_pr[0]

(1.784469674289808, 1.77604570919968, 1.7724381959297646)

In [269]:
g_ic[0]/h_ic[0,0],dq_pr[0],g_ic[0]/h_ic[0,0]+dq_pr[0]

(0.06304673043868449, -0.07147069552881277, -0.008423965090128285)

In [233]:
#dq_pr=-np.linalg.inv(h_ic)@g_ic
h_ic_inv=np.linalg.inv(h_ic)
(-h_ic_inv@g_ic)[0]

-0.07147069552881277

In [232]:
h_ic_inv[:,0],h_ic_inv[:,0].dot(g_ic),h_ic_inv[0,0]*g_ic[0],h_ic_inv[1:,0].dot(g_ic[1:])

(array([ 2.023,  0.041, -0.36 ]),
 0.07187326463857507,
 0.0633266979455222,
 0.008546566693052867)

In [103]:
(-h_ic@dq_mb0)[2],g_ic[2]

(-0.020438207901431665, -0.020171759719946218)

In [104]:
h_ic[:,2],(-h_ic@dq_mb0)[2]-g_ic[2]

(array([0.037, 0.037, 0.214]), -0.0002664481814854472)

In [105]:
ddq_ma=np.asarray([0,0,((-h_ic@dq_mb0)[2]-g_ic[2])/h_ic[2,2]])
ddq_ma

array([ 0.   ,  0.   , -0.001])

In [106]:
(-h_ic@(dq_mb0+ddq_ma))[2],g_ic[2]

(-0.020171759719946215, -0.020171759719946218)

In [107]:
s.future.q+(dq_mb0+ddq_ma)

array([1.776, 1.776, 1.842])

In [108]:
mpa(g_ic[2],h_ic[2,2],s.future.q[2],10000)[0]

1.8185534082643486

In [109]:
g_ic[2]/h_ic[2,2]

-0.09416976554063622

In [110]:
s.future.q

array([1.844, 1.844, 1.724])

In [113]:
g0@np.asarray([0,1.2,1])/norm(np.asarray([0,1.2,1]))

array([-0.037,  0.006,  0.031])

In [115]:
B@B.T

array([[ 2.   , -0.153, -0.536],
       [-0.153,  2.   , -0.536],
       [-0.536, -0.536,  1.266]])

In [116]:
h0,B.T@h_ic@B

(array([[ 4.779e-02,  1.875e-16, -8.111e-17, -2.389e-02, -1.144e-16,
         -1.504e-16, -2.389e-02, -7.309e-17,  2.316e-16],
        [ 1.875e-16,  5.973e-01,  2.661e-15, -1.035e-16, -2.987e-01,
          2.355e-01, -8.399e-17, -2.987e-01, -2.355e-01],
        [-8.111e-17,  2.151e-15,  4.865e-01, -4.588e-17,  1.773e-01,
         -2.433e-01,  1.270e-16, -1.773e-01, -2.433e-01],
        [-2.389e-02, -1.035e-16, -4.588e-17,  1.789e-02,  5.144e-17,
          1.003e-16,  6.004e-03,  5.208e-17, -5.446e-17],
        [-1.144e-16, -2.987e-01,  1.773e-01,  5.144e-17,  3.394e-01,
         -2.064e-01,  6.298e-17, -4.070e-02,  2.911e-02],
        [-1.504e-16,  2.355e-01, -2.433e-01,  1.003e-16, -2.064e-01,
          2.285e-01,  5.010e-17, -2.911e-02,  1.475e-02],
        [-2.389e-02, -8.399e-17,  1.270e-16,  6.004e-03,  6.298e-17,
          5.010e-17,  1.789e-02,  2.101e-17, -1.771e-16],
        [-7.309e-17, -2.987e-01, -1.773e-01,  5.208e-17, -4.070e-02,
         -2.911e-02,  2.101e-17,  3.394e-0

# Understand the building process for $B_{ij}=\frac{\partial q_i}{\partial x_j}$

In [117]:
# start from B matrix construction  
def B_matrix(self, geom):
        geom = geom.supercell()
        B = np.zeros((len(self), len(geom), 3))
        for i, coord in enumerate(self):   # i=index, internal coordinate
            _, grads = coord.eval(geom.coords, grad=True)
            idx = [k % len(geom) for k in coord.idx] #list with atom ideces
            for j, grad in zip(idx, grads): # j is an atom index 
                B[i, j] += grad    # is built at first a matrix of 3n vectors 
        return B.reshape(len(self), 3 * len(geom)) # in the end is reshaped to N.IC x 3 N.atm 

In [118]:
idxl=[k % len(s.geom) for k in s.coords._coords[1].idx]
print(*zip(idxl,s.coords._coords[0].eval(s.geom.coords, grad=True)[1]))

(0, array([ 0.   ,  0.759, -0.651])) (2, array([-0.   , -0.759,  0.651]))


In [119]:
s.coords._coords[1].idx

(0, 2)

In [120]:
print(*enumerate(s.coords))

(0, Bond(0, 1, weak=0)) (1, Bond(0, 2, weak=0)) (2, Angle(1, 0, 2, weak=0))


In [121]:
s.coords._coords[0].eval(s.geom.coords, grad=True)

(1.8439088914585773,
 [array([ 0.   ,  0.759, -0.651]), array([-0.   , -0.759,  0.651])])

In [122]:
print(inspect.getsource(s.coords._coords[0].eval)) # angstrom=ang2bohr
# the bond gradient is difference between the two coordinates, over the norm

    def eval(self, coords, grad=False):
        v = (coords[self.i] - coords[self.j]) * angstrom
        r = norm(v)
        if not grad:
            return r
        return r, [v / r, -v / r]



 # Trying to obtain the tensor $B'_{ijk}= \frac {\partial^2 q_i}{\partial x_j\partial x_k}$

In [154]:
def bond_2deriv(bond,coords):
    v = (coords[bond.i] - coords[bond.j]) * ang2bohr
    r = norm(v)
    fd=np.hstack((v / r,-v / r))
    bp=np.einsum("i,j->ij",fd,fd)
    bp-=np.eye(6)
    bp[0,3]+=1
    bp[3,0]+=1
    bp[1,4]+=1
    bp[4,1]+=1
    bp[2,5]+=1
    bp[5,2]+=1
    bp/=-r
    return  bp

In [155]:
Bp=bond_2deriv(s.coords._coords[0],s.geom.coords),bond_2deriv(s.coords._coords[1],s.geom.coords)

In [156]:
s.coords._coords[1],s.coords._coords[0]

(Bond(0, 2, weak=0), Bond(0, 1, weak=0))

In [157]:
Bp

(array([[ 0.54233, -0.     , -0.     , -0.54233, -0.     , -0.     ],
        [-0.     ,  0.22969,  0.26797, -0.     , -0.22969, -0.26797],
        [-0.     ,  0.26797,  0.31264, -0.     , -0.26797, -0.31264],
        [-0.54233, -0.     , -0.     ,  0.54233, -0.     , -0.     ],
        [-0.     , -0.22969, -0.26797, -0.     ,  0.22969,  0.26797],
        [-0.     , -0.26797, -0.31264, -0.     ,  0.26797,  0.31264]]),
 array([[ 0.54233, -0.     , -0.     , -0.54233, -0.     , -0.     ],
        [-0.     ,  0.22969, -0.26797, -0.     , -0.22969,  0.26797],
        [-0.     , -0.26797,  0.31264, -0.     ,  0.26797, -0.31264],
        [-0.54233, -0.     , -0.     ,  0.54233, -0.     , -0.     ],
        [-0.     , -0.22969,  0.26797, -0.     ,  0.22969, -0.26797],
        [-0.     ,  0.26797, -0.31264, -0.     , -0.26797,  0.31264]]))

In [158]:
s.coords._coords[0]

Bond(0, 1, weak=0)

In [159]:
v=np.asarray([1,2,3])

In [160]:
np.einsum("i,j->ij",v,v)

array([[1, 2, 3],
       [2, 4, 6],
       [3, 6, 9]])

In [161]:
s.coords._coords[0].__class__==coords.Bond

True

$$B^T H_qB + B'^Tg_q = H_x $$
$$ H_q = B_{inv}^T(H_x - B'^Tg_q)B_{inv}$$

In [162]:
B_inv.T@h0@B_inv,h_ic

(array([[ 0.50091, -0.00357,  0.03732],
        [-0.00357,  0.50091,  0.03732],
        [ 0.03732,  0.03732,  0.21421]]),
 array([[ 0.50091, -0.00357,  0.03732],
        [-0.00357,  0.50091,  0.03732],
        [ 0.03732,  0.03732,  0.21421]]))

In [168]:
BpG=np.zeros((9,9))
BpG[:6,:6]+=Bp[0]*g_ic[0]
BpG[:3,:3]+=Bp[1][:3,:3]*g_ic[1]
BpG[6:,6:]+=Bp[1][3:,3:]*g_ic[1]
BpG[6:,:3]+=Bp[1][3:,:3]*g_ic[1]
BpG[:3,6:]+=Bp[1][:3,3:]*g_ic[1]

In [169]:
g_ic

array([ 0.0313 ,  0.0313 , -0.02017])

In [170]:
BpG

array([[ 3.39458e-02,  0.00000e+00,  0.00000e+00, -1.69729e-02,
         0.00000e+00,  0.00000e+00, -1.69729e-02,  0.00000e+00,
         0.00000e+00],
       [ 0.00000e+00,  1.43770e-02,  6.93889e-16,  0.00000e+00,
        -7.18851e-03, -8.38660e-03,  0.00000e+00, -7.18851e-03,
         8.38660e-03],
       [ 0.00000e+00,  6.93889e-16,  1.95687e-02,  0.00000e+00,
        -8.38660e-03, -9.78437e-03,  0.00000e+00,  8.38660e-03,
        -9.78437e-03],
       [-1.69729e-02,  0.00000e+00,  0.00000e+00,  1.69729e-02,
         0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
         0.00000e+00],
       [ 0.00000e+00, -7.18851e-03, -8.38660e-03,  0.00000e+00,
         7.18851e-03,  8.38660e-03,  0.00000e+00,  0.00000e+00,
         0.00000e+00],
       [ 0.00000e+00, -8.38660e-03, -9.78437e-03,  0.00000e+00,
         8.38660e-03,  9.78437e-03,  0.00000e+00,  0.00000e+00,
         0.00000e+00],
       [-1.69729e-02,  0.00000e+00,  0.00000e+00,  0.00000e+00,
         0.00000e+00,  0.00000

In [186]:
np.set_printoptions(precision=3)

In [174]:
B_inv.T@(h0-BpG)@B_inv,h_ic

(array([[ 0.49913, -0.00178,  0.03732],
        [-0.00178,  0.49913,  0.03732],
        [ 0.03732,  0.03732,  0.18535]]),
 array([[ 0.50091, -0.00357,  0.03732],
        [-0.00357,  0.50091,  0.03732],
        [ 0.03732,  0.03732,  0.21421]]))

# Get B' for angles !!!


In [204]:
geom0=s.geom.copy()

In [176]:
s.coords.B_matrix(s.geom)

array([[ 0.     ,  0.75926, -0.65079,  0.     , -0.75926,  0.65079,
         0.     ,  0.     ,  0.     ],
       [ 0.     , -0.75926, -0.65079,  0.     ,  0.     ,  0.     ,
         0.     ,  0.75926,  0.65079],
       [ 0.     ,  0.     ,  0.82353,  0.     , -0.35294, -0.41176,
         0.     ,  0.35294, -0.41176]])

In [207]:
geom0.coords,geom0.coords.flatten()#.reshape(3,3)


(array([[ 0.   ,  0.   ,  0.   ],
        [ 0.   , -0.741,  0.635],
        [ 0.   ,  0.741,  0.635]]),
 array([ 0.   ,  0.   ,  0.   ,  0.   , -0.741,  0.635,  0.   ,  0.741,
         0.635]))

In [201]:
s.coords.B_matrix(geom0)

array([[nan, nan, nan, nan, nan, nan,  0.,  0.,  0.],
       [nan, nan, nan,  0.,  0.,  0., nan, nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan]])

In [181]:
geom0.coords

array([[ 0.     ,  0.     ,  0.     ],
       [ 0.     , -0.74085,  0.63601],
       [ 0.     ,  0.74085,  0.63501]])

In [222]:
bms=[]
for i in range(geom0.coords.flatten().shape[0]):
    a=geom0.coords.flatten()
    geom1=geom0.copy()
    a[i]+=.001*bohr2ang
    a=a.reshape(3,3)
    geom1.coords=a
    bms.append((s.coords.B_matrix(geom1)-B)*1000)

In [224]:
bms_arr=np.asarray(bms)

In [228]:
BpG2=np.einsum('ijk,j->ik',bms,g_ic)

In [230]:
h_ic=B_inv.T@(h0-BpG2)@B_inv